# DADOS DE CARGA DE ENERGIA DIÁRIA

**Descrição do Dado:** Dados de carga por subsistema numa data de referência em base diária.

| Descrição | Código | Tipo de Dado | Formato | Permite valor nulo | Permite valor zerado | Permite valor negativo |
| --- | --- | --- | --- | --- | --- | --- |
| Código do Subsistema | `id_subsistema` | STRING | 3 POSIÇÕES | Não | - | - |
| Nome do Subsistema | `nom_subsistema` | STRING | 60 POSIÇÕES | Não | - | - |
| Data de referência | `din_instante` | DATETIME | `YYY-MM-DD HH:MM:SS` | Não | - | - |
| Valor da Carga de Energia (MWmed) | `val_cargaenergiamwmed` | FLOAT | - | Não | Não | Não |

<small>Última edição em `01-05-2023`.</small>

In [2]:
import pandas
import os
import re
import subprocess
import requests
import json

from selenium import webdriver
from splinter import Browser
from http import HTTPStatus
from http.client import responses

In [3]:
__DADOS_ABERTOS_ONS_CARGA_URL = "https://dados.ons.org.br/dataset/carga-energia"
__DOWNLOAD_DIR = os.path.join(os.getcwd(), "data")
__REGEX_PATTERN_FILENAME = r'(CARGA_ENERGIA_)[0-9]{4,}'

if not os.path.exists(__DOWNLOAD_DIR):
    os.mkdir(__DOWNLOAD_DIR)

__GECKODRIVER_OPTIONS = webdriver.FirefoxOptions()
# __GECKODRIVER_OPTIONS.add_argument("--start-maximized")
__GECKODRIVER_OPTIONS.binary_location = r'C:\Webdriver\bin\Firefox\geckodriver.exe'
__BROWSER_OPTIONS = {
    'driver_name': 'firefox',
    'headless': True,
    'options': __GECKODRIVER_OPTIONS
}

In [4]:
from typing import *
from traceback import *

def __format_stacktrace(text: str, **kwargs: Dict[str, Any]) -> str:
    message: str = text
    args: Dict[str, Any] = kwargs.pop('args', {})
    if args:
        for key, value in args.items():
            message += f"\n- {key}: {value}"
    return message

In [5]:
browser = Browser(**__BROWSER_OPTIONS)
browser.visit(__DADOS_ABERTOS_ONS_CARGA_URL)
csv_urls = [item['href'] for item in browser.find_by_xpath('//a[contains(@href, ".csv")]')]
browser.quit()

In [6]:
print(f'[INFO] Downloading CSV files to path "{__DOWNLOAD_DIR}":')

for csv_url in csv_urls:
    csv_filename = re.search(f'{__REGEX_PATTERN_FILENAME}(.csv)', csv_url).group()
    response = requests.get(csv_url)
    response_status = response.status_code
    if response_status == HTTPStatus.OK:
        with open(os.path.join(__DOWNLOAD_DIR, csv_filename), 'w') as csv_file:
            csv_file.write(response.content.decode('utf8'))
            print(
                f'\t* [{response_status} - {responses[response_status]}] \
                Successfully downloaded file: "{csv_filename}"'
             )
    else:
        print(
            f'* [{response_status} - {responses[response_status]}] \
            Unable to download file: "{csv_filename}"'
         )

[INFO] Downloading CSV files to path "C:\Users\nadio\Documents\GitHub\tcc-nadio-pontes\notebooks\data":
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2000.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2001.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2002.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2003.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2004.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2005.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2006.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2007.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2008.csv"
	* [200 - OK]                 Successfully downloaded file: "CARGA_ENERGIA_2009.csv"
	* [200 - OK]                 Successfully dow

In [7]:
csv_files_path = [
    os.path.join(__DOWNLOAD_DIR, file)\
    for file in os.listdir(__DOWNLOAD_DIR) if file.endswith('.csv')
]

for csv_file_path in csv_files_path:
    carga_df = pandas.read_csv(csv_file_path, sep=';')
    subsistemas = carga_df['id_subsistema'].unique()
    nome_subsistemas = carga_df['nom_subsistema'].unique()
    cargas = []
    
    for subsistema, nome_subsistema in zip(subsistemas, nome_subsistemas):
        cargas.append({
            'id': subsistema,
            'nome': nome_subsistema,
            'registros': [
                # "data": DATETIME -- 'din_instante'
                # "carga" FLOAT -- 'val_cargaenergiamwmed'
            ]
        })
    
    datas = carga_df['din_instante'].unique()
    json_filename = f'{re.search(__REGEX_PATTERN_FILENAME, csv_file_path).group()}.json'
    
    for data in datas:
        df = carga_df.loc[carga_df['din_instante'] == data]
        for carga in cargas:
            carga_id, carga_nome, carga_registros = carga
            valor_carga = df.loc[df['id_subsistema'] == carga[carga_id]]\
                ['val_cargaenergiamwmed']
            energia = 0
            try:
                if len(valor_carga) > 0:
                    energia = float(valor_carga.iloc[0])
                    
                    if pandas.isna(energia):
                        continue
                        
                    carga[carga_registros].append({
                        'data': data,
                        'energia': energia
                    })
                else:
                    continue
            except:
                print(
                    __format_stacktrace(
                        text=f"[FATAL] Unexpected data processing behavior.",
                        args={
                            'File': json_filename,
                            'Data': df.to_json(),
                            'Data (type)': type(valor_carga),
                            'Stacktrace': format_exc()
                        }
                    )
                )
    
    with open(os.path.join(__DOWNLOAD_DIR, json_filename), 'w') as json_file:
        json_file.write(json.dumps(cargas, indent=True))
        print(
            f'[INFO] Successfully created file: \
            "{json_filename}"'
        )

[INFO] Successfully created file:             "CARGA_ENERGIA_2000.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2001.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2002.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2003.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2004.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2005.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2006.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2007.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2008.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2009.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2010.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2011.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_2012.json"
[INFO] Successfully created file:             "CARGA_ENERGIA_201